In [1]:
#import os
#import sys
import numpy as np

#from astropy.io import fits

#import photutils
#from photutils import EllipticalAperture

#import matplotlib.pyplot as plt
#%matplotlib inline

# use nicer set of plot parameters (from astropy www)
#from astropy.visualization import astropy_mpl_style
#plt.style.use(astropy_mpl_style)

import statsmodels.api as sm

from scipy.interpolate import interp1d

/Users/roedigerj/anaconda/envs/astro/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# ISSUES
#  applying trend cast wrt CoG magnitudes to Galfit magnitudes

In [3]:
# READ TOTAL ERRORS FOR GALAXIES HAVING DIRECT ERROR ESTIMATES

f = open('skyErrorEstimates_stdDev_wSkyInPoissonTermAndCalErr.txt', 'r')
lines = f.readlines()
f.close()

ngvsIDs_train = np.array([line.strip().split()[0] for line in lines[1:]])
bands_train = np.array([line.strip().split()[1] for line in lines[1:]])
errs_train = np.array([float(line.strip().split()[10]) for line in lines[1:]])

In [5]:
# READ GALFIT MAGNITUDES FOR EVERY GALAXY FROM FULL SURVEY

f = open('mags_ngvsFull_err=deltaMag.txt', 'r')
lines = f.readlines()
f.close()

ngvsIDs_full = np.array([line.strip().split()[0] for line in lines])

mags_full = np.zeros([len(lines), 5])
mags_full[:, 0] = [float(line.strip().split()[13]) for line in lines]
mags_full[:, 1] = [float(line.strip().split()[11]) for line in lines]
mags_full[:, 2] = [float(line.strip().split()[15]) for line in lines]
mags_full[:, 3] = [float(line.strip().split()[17]) for line in lines]
mags_full[:, 4] = [float(line.strip().split()[19]) for line in lines]

In [ ]:
# CREATE SEPERATE ARRAY OF MAGNITUDES FOR TRAINING SET BY CROSS-MATCHING FULL SURVEY WITH PILOT FIELD

magsList = []
for ngvsID_train, band_train in zip(ngvsIDs_train, bands_train):
    mask = (ngvsIDs_full == ngvsID_train)
    if np.sum(mask) == 0:
        continue
        
    if band_train == 'U':
        magsList.append(mags_full[mask, 0][0])
    elif band_train == 'G':
        magsList.append(mags_full[mask, 1][0])
    elif band_train == 'R':
        magsList.append(mags_full[mask, 2][0])
    elif band_train == 'I':
        magsList.append(mags_full[mask, 3][0])
    elif band_train == 'Z':
        magsList.append(mags_full[mask, 4][0])
        
mags_train = np.array(magsList)

In [10]:
# ESTIMATE ERRORS IN GALFIT PHOTOMETRY FOR FULL NGVS GALAXY CATALOGUE B/O TRAINING SET (FROM PILOT FIELD)

minErr = [0.0283, 0.0141, 0.0141, 0.0141, 0.0224]

magErrs_full = -100.0 * np.ones(mags_full.shape)

for j, band in enumerate(['U', 'G', 'R', 'I', 'Z']):
    
    # grab mags and errors for band, and stash in single array (sorted by mag)
    selection = (bands_train == band)
    data = np.stack([mags_train[selection], errs_train[selection]], axis = 1)
    data = data[data[:, 0].argsort()]
    print (data.shape)
    data = data[data[:, 0] > 0.]
    print (data.shape)
    
    # replace errors lying below value for calibration error in band
    mask = (data[:, 1] < minErr[j])
    #data[mask, 1] = minErr[j]
    
    # determine LOWESS trend between selected mags and errors (write to file, omitting repeated values)
    lowess = sm.nonparametric.lowess(data[:, 1], data[:, 0], frac = 0.3, it = 3)
    f = open('lowess_totErrsVsMags_' + band + '_x=galfit.txt', 'w')
    
    x, indxs = np.unique(lowess[:, 0], return_index = True)
    y = lowess[indxs, 1]
    for i in range(lowess.shape[0]):
        if i in indxs:
            f.write('%8.4f %9.5f\n' % (lowess[i, 0], lowess[i, 1]))
    f.close()
    
    # interpolate LOWESS solution and predict magnitude errors therefrom
    mask = (mags_full[:, j] > 0.)
    func = interp1d(x, y, kind = 'cubic', bounds_error = False, fill_value = "extrapolate")
    magErrs_full[mask, j] = func(mags_full[mask, j])
    
    # fix erroneous values caused by extrapolation
    mask = ((mags_full[:, j] < lowess[0, 0]) & (mags_full[:, j] > 0.))
    magErrs_full[mask, j] = lowess[0, 1]
    
    mask = (mags_full[:, j] > lowess[-1, 0])
    magErrs_full[mask, j] = lowess[-1, 1]
    
    # add scatter to predicted errors
    count = 0
    for i in range(mags_full.shape[0]):
        if mags_full[i, j] < 0.:
            continue
            
        # find 10 closest neighours from pilot field to galaxy in mag-space 
        idx = np.searchsorted(data[:, 0], mags_full[i, j])
        argMin = np.max([idx - 5, 0])
        argMax = np.min([argMin + 10, data.shape[0]])
        
        # find number of neighbours from pilot field lying within X mag of galaxy
        mask = (np.abs(data[:, 0] - mags_full[i, j]) <= 0.5)
        
        # use fixed or adaptive bin to compute standard deviation
        if np.sum(mask) < 10:
            #scratch = data[argMin:argMax, 1] - lowess[argMin:argMax, 1]
            scratch = np.log10(data[argMin:argMax, 1]) - np.log10(lowess[argMin:argMax, 1])
            count += 1
        else:
            #scratch = data[mask, 1] - lowess[mask, 1]
            scratch = np.log10(data[mask, 1]) - np.log10(lowess[mask, 1])
            
        delta = np.random.normal(loc = np.log10(magErrs_full[i, j]), 
                                 scale = np.std(scratch, ddof = 1, dtype = np.float64))
        #magErrs_full[i, j] = delta
        magErrs_full[i, j] = 10.0 ** delta
        
    # replace errors lying below value for calibration error in band
    mask = ((magErrs_full[:, j] < minErr[j]) & (mags_full[:, j] > 0.))
    magErrs_full[mask, j] = minErr[j]
    
    mask = ((magErrs_full[:, j] < 10.0 ** (0.2 * (mags_full[:, j] - 30.) + 0.0357)) & \
            (mags_full[:, j] > 0.))
    magErrs_full[mask, j] = 10.0 ** (0.2 * (mags_full[mask, j] - 30.) + 0.0357)

(401, 2)
(364, 2)
(402, 2)
(363, 2)
(401, 2)
(355, 2)
(402, 2)
(362, 2)
0.0272963599486 -1.6884389597591474 [-0.26175414 -0.22392058  0.33474983 -0.11841323 -0.05688174  0.02078749
 -0.31294687 -0.16105918 -0.2950589  -0.17412597]
(401, 2)
(351, 2)


In [12]:
# WRITE TABLE OF GALFIT MAGNITUDES AND ESTIMATED TOTAL ERRORS TO FILE

f = open('mags_galfit_ngvsFull_err=sky+Gxy.txt', 'w')
for i in range(mags_full.shape[0]):
    string = '%s' % (ngvsIDs_full[i])
    for j in range(0, 5):
        string += '%12.4f %12.5f' % (mags_full[i, j], magErrs_full[i, j])
        
    f.write(string + '\n')
f.close()